# Stock purchase recommendations with Machine Learning

In [268]:
import pandas_datareader.data as web
import pandas as pd
import numpy as np
import talib as ta
import matplotlib.pyplot as plt
from tqdm import tqdm # progress bar

from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

from sklearn.ensemble import RandomForestClassifier

In [290]:
pd.set_option('display.max_columns', 150)

In [269]:
# read in the Quandl.com API key - saved in separate file to keep it private

filepath = 'confidential-API-key.txt'  
with open(filepath) as fp:  
   my_Quandl_API_key = fp.readline()

## Load stock data from Quandl

In [270]:
def load_stock_data(ticker, start_date, end_date, Quandl_API_key=my_Quandl_API_key):
    '''
    Downloads stock data from Quandl, drop some columns, resort datafram, and return
    Inputs:
        ticker - a stock ticker symbol (needs to be valid - no error checking implemented)
        start_date - first date of stock prices
        end_date - last date of stock prices
        Quandl_API_key - string with valid API key for Quandl.com data queries
    Outputs:
        stock_data - DataFrame with stock price data, sorted in ascending date order
    '''
    
    # download data from Quandl with Pandas Datareader
    stock_data = web.DataReader(name=symbol, data_source='quandl', start=start_date, end=end_date, access_key=Quandl_API_key)
    
    # need ascending index for the TA-lib indicators to work properly
    stock_data.sort_index(inplace=True)
    
    # keep only the columns with adjusted data to eliminate any issues due to stock splits
    stock_data = stock_data[['AdjVolume', 'AdjOpen', 'AdjHigh', 'AdjLow', 'AdjClose']]
    
    # DataFram has a second level column index with the stock ticker - not needed so drop it
    stock_data.columns = stock_data.columns.droplevel(1)
    
    # add row index (highest number is most recent date) - this will be used to later restack the rows for feature matrix
    stock_data['row_index'] = range(0, stock_data.shape[0])
    
    return stock_data    

In [271]:
# test loading stock data:

start = '2013-01-01'
end = '2019-03-30'
symbol = ['AAPL']

df = load_stock_data(symbol, start, end)

df.head()

Attributes,AdjVolume,AdjOpen,AdjHigh,AdjLow,AdjClose,row_index
Date,,,,,,
2013-01-02,140129500.0,71.816894,71.969911,70.236149,71.195748,0
2013-01-03,88241300.0,71.046621,71.278740,70.154453,70.296565,1
2013-01-04,148583400.0,69.631213,69.847109,68.187094,68.338996,2
2013-01-07,121039100.0,67.690619,68.637250,66.808825,67.937002,3
2013-01-08,114676800.0,68.625579,68.973109,67.593362,68.119845,4


In [272]:
df.shape

(1316, 6)

## Define Target column: profitability of trade

The goal is the define a column "setup_for_profitable_trade" based on a trading strategy. The data will be analyzed for this trade setup:

Run the analysis in the evening after market close. That day's row of market data will then be either deemed as profitable or not for a trade to be initiated and closed in the following days. This setup of the data ensures no lookahead is happening.

## Trading Strategy:

1. After market close on day N decide if buying stock at market Open on day N+1
2. Open position: submit market order for day N+1 prior to market open
3. Close position: submit sell order for market open for day N+2

Threshold to decide to buy the stock: expected profit from open to close: 0.5% profit

Store in row N if this trade was deemed profitable.

In [273]:
def add_flag_for_profitable_trade_setups(df, delete_interim_calculation_cols=True):
    '''
    Adds column to dataframe that identifies profitable trading setup
    Inputs:
        df - dataframe with stock data
        delete_interim_calculation_cols - if false: keep the interim calculations - good for debugging
    Outputs:
        df - dataframe with additional columns
    
    '''
    
    # trade strategy: after market close on day N, set a buy at market open on day N+1 and sell at market close on day N+1
    # for expected gain of at least 0.5%
    profitability_threshold = 0.005

    # use helper columns to calculate profit
    df['strategy_open_price'] = df['AdjOpen'].shift(-1) # AdjOpen from day N+1
    df['strategy_close_price'] = df['AdjOpen'].shift(-2) # AdjOpen from day N+2
    df['strategy_profit_dollars'] = df['strategy_close_price'] - df['strategy_open_price']
    df['strategy_profitability'] = df['strategy_profit_dollars'] / df['strategy_open_price']

    # use categorical field to encode "setup_for_succesful_trade": 1=yes, 0=no
    df['setup_for_profitable_trade'] = df['strategy_profitability'] >= profitability_threshold
    
    if delete_interim_calculation_cols:
        df = df.drop(columns=['strategy_open_price', 'strategy_close_price', 'strategy_profit_dollars', 'strategy_profitability'])
    
    # drop any rows that have NaNs in them. especially the last few rows will have NaNs because the profitability calc
    # looks into the future beyond the last row. these rows need to get deleted - the profitability is not defined there
    df = df.loc[df.notnull().all(axis=1), :]
    
    return df

In [274]:
# testing: show calculations:
df = add_flag_for_profitable_trade_setups(df, False)
df.tail(10)

Attributes,AdjVolume,AdjOpen,AdjHigh,AdjLow,AdjClose,row_index,strategy_open_price,strategy_close_price,strategy_profit_dollars,strategy_profitability,setup_for_profitable_trade
Date,,,,,,,,,,,
2018-03-12,32055405.0,180.29,182.39,180.2100,181.720,1304,182.59,180.32,-2.27,-0.012432,False
2018-03-13,31168404.0,182.59,183.50,179.2400,179.970,1305,180.32,178.50,-1.82,-0.010093,False
2018-03-14,29075469.0,180.32,180.52,177.8100,178.440,1306,178.50,178.65,0.15,0.000840,False
2018-03-15,22584565.0,178.50,180.24,178.0701,178.650,1307,178.65,177.32,-1.33,-0.007445,False
2018-03-16,36836456.0,178.65,179.12,177.6200,178.020,1308,177.32,175.24,-2.08,-0.011730,False
2018-03-19,32804695.0,177.32,177.47,173.6600,175.300,1309,175.24,175.04,-0.20,-0.001141,False
2018-03-20,19314039.0,175.24,176.80,174.9400,175.240,1310,175.04,170.00,-5.04,-0.028793,False
2018-03-21,35247358.0,175.04,175.09,171.2600,171.270,1311,170.00,168.39,-1.61,-0.009471,False
2018-03-22,41051076.0,170.00,172.68,168.6000,168.845,1312,168.39,168.07,-0.32,-0.001900,False


In [275]:
# without verbose results:
df = add_flag_for_profitable_trade_setups(df)
df.tail(10)

Attributes,AdjVolume,AdjOpen,AdjHigh,AdjLow,AdjClose,row_index,setup_for_profitable_trade
Date,,,,,,,
2018-03-12,32055405.0,180.29,182.39,180.2100,181.720,1304,False
2018-03-13,31168404.0,182.59,183.50,179.2400,179.970,1305,False
2018-03-14,29075469.0,180.32,180.52,177.8100,178.440,1306,False
2018-03-15,22584565.0,178.50,180.24,178.0701,178.650,1307,False
2018-03-16,36836456.0,178.65,179.12,177.6200,178.020,1308,False
2018-03-19,32804695.0,177.32,177.47,173.6600,175.300,1309,False
2018-03-20,19314039.0,175.24,176.80,174.9400,175.240,1310,False
2018-03-21,35247358.0,175.04,175.09,171.2600,171.270,1311,False
2018-03-22,41051076.0,170.00,172.68,168.6000,168.845,1312,False


In [276]:
df['setup_for_profitable_trade'].sum()

497

## Feature Engineering

In [277]:
#split df into features and target (column setup_for_profitable_trade)
df_X_base_data = df.drop(columns=['setup_for_profitable_trade'])

### Feature Engineering - Add Technical Analysis Indicators

In [278]:
def add_TALib_indicator(df, attribute, indicator_func, *args):
    '''
    Adds a column to a dataframe:
        column name is the name of the technical indicator as specified by indicator_func
        column content is the function calculated on the attribute column
    Example: add_TALib_indicator(df, 'AdjClose', ta.RSI, 14) creates a new column called RSI with 
             the 14 day RSI of the values of the column 'AdjClose'
    Inputs:
        df - dataframe - needs to be sorted in date ascending order
        attribute - column name to be used in TA-Lib calculation
        indicator_func - name of a TA-Lib function
        *args - optional parameters for indicator_func
        
    Oupputs:
        df - datarame with new column added
        func_name - name of the new colunm
    
    '''
    # get the name of the indicator from TA-Lib
    func_name = attribute + indicator_func.__name__
    
    # add new column, calculated based on attribute column
    df.loc[:, func_name] = indicator_func(df.loc[:, attribute].values, *args)
    
    return df, func_name

In [279]:
df_X_base_data, indicator_name = add_TALib_indicator(df_X_base_data, 'AdjClose', ta.RSI, 14)

df_X_base_data.tail()

Attributes,AdjVolume,AdjOpen,AdjHigh,AdjLow,AdjClose,row_index,AdjCloseRSI
Date,,,,,,,
2018-03-19,32804695.0,177.32,177.47,173.66,175.300,1309,50.310859
2018-03-20,19314039.0,175.24,176.80,174.94,175.240,1310,50.179395
2018-03-21,35247358.0,175.04,175.09,171.26,171.270,1311,42.302794
2018-03-22,41051076.0,170.00,172.68,168.60,168.845,1312,38.343555
2018-03-23,40248954.0,168.39,169.92,164.94,164.940,1313,32.989187


In [280]:
df_X_base_data.head(16)
#confirms NaN for RSI on top for first 14 dates since it is a 14 day RSI

Attributes,AdjVolume,AdjOpen,AdjHigh,AdjLow,AdjClose,row_index,AdjCloseRSI
Date,,,,,,,
2013-01-02,140129500.0,71.816894,71.969911,70.236149,71.195748,0,NaN
2013-01-03,88241300.0,71.046621,71.278740,70.154453,70.296565,1,NaN
2013-01-04,148583400.0,69.631213,69.847109,68.187094,68.338996,2,NaN
2013-01-07,121039100.0,67.690619,68.637250,66.808825,67.937002,3,NaN
2013-01-08,114676800.0,68.625579,68.973109,67.593362,68.119845,4,NaN
2013-01-09,101901100.0,67.755456,68.080345,66.911269,67.055209,5,NaN
2013-01-10,150286500.0,68.539993,68.562038,66.850321,67.886429,6,NaN
2013-01-11,87626700.0,67.560943,68.121141,67.304186,67.470170,7,NaN
2013-01-14,183551900.0,65.185288,65.810324,64.644541,65.064689,8,NaN


### Feature Engineering - Changing from actual values to percentage changes

In [281]:
def feat_eng_changes_values_to_change(df, cols_set_vals_to_change, delete_original_cols=True):
    '''
    Instead of the actual values in some columns, we care about the change from one day to the next.
    This function calculates that change for the given columns and then either keeps or drops (default) the origianl columns
    Input:
        df - a dataframe
        cols_set_vals_to_change - names of columns to work on.
        delete_original_cols - keep or delete original columns
    Output:
        df - dataframe with new columns added. the value in row N is now the change from row N-1 to row N (instead of the actual values)
    '''    

    # calculate the change from row N-1 to row N
    df_chg_cols = (df[cols_set_vals_to_change] / df[cols_set_vals_to_change].shift(1) - 1)

    # add suffix to the column names
    df_chg_cols = df_chg_cols.add_suffix('_chg')

    # join the data onto the original data fram
    df = df.join(df_chg_cols)

    if delete_original_cols:
        # drop the original columns
        df = df.drop(columns=cols_set_vals_to_change)
        
    return df

cols_set_vals_to_change = ['AdjVolume', 'AdjOpen', 'AdjLow', 'AdjHigh', 'AdjClose']
df_X_base_data = feat_eng_changes_values_to_change(df_X_base_data, cols_set_vals_to_change, delete_original_cols=False)

df_X_base_data.tail()

Attributes,AdjVolume,AdjOpen,AdjHigh,AdjLow,AdjClose,row_index,AdjCloseRSI,AdjVolume_chg,AdjOpen_chg,AdjLow_chg,AdjHigh_chg,AdjClose_chg
Date,,,,,,,,,,,,
2018-03-19,32804695.0,177.32,177.47,173.66,175.300,1309,50.310859,-0.109450,-0.007445,-0.022295,-0.009212,-0.015279
2018-03-20,19314039.0,175.24,176.80,174.94,175.240,1310,50.179395,-0.411242,-0.011730,0.007371,-0.003775,-0.000342
2018-03-21,35247358.0,175.04,175.09,171.26,171.270,1311,42.302794,0.824960,-0.001141,-0.021036,-0.009672,-0.022655
2018-03-22,41051076.0,170.00,172.68,168.60,168.845,1312,38.343555,0.164657,-0.028793,-0.015532,-0.013764,-0.014159
2018-03-23,40248954.0,168.39,169.92,164.94,164.940,1313,32.989187,-0.019540,-0.009471,-0.021708,-0.015983,-0.023128


### Feature Engineering - Reshaping the prior day data into ML-ready features matrix
#### --- build pieces for the dataframe section unpivoting ---
Goal: pull out n_features rows, flatten, and build meaningful column names that indicate how many days' back the data is from

In [282]:
# parameter for how many days of history to include in feature list
n_days_features = 5

# total lenght of df
n_data_points = df.shape[0]

In [283]:
i = 505
# pull out n_days_features of rows from current position
df_extract = df_X_base_data.iloc[i-n_days_features:i, :].copy()

# change the index the be "days into the past" - eg current day is 0, prior day is -1, ...
df_extract.loc[:, 'row_index'] = range(-n_days_features+1, 1)

# make this the new index
df_extract.set_index('row_index', inplace=True)

df_extract

Attributes,AdjVolume,AdjOpen,AdjHigh,AdjLow,AdjClose,AdjCloseRSI,AdjVolume_chg,AdjOpen_chg,AdjLow_chg,AdjHigh_chg,AdjClose_chg
row_index,,,,,,,,,,,
-4,33720951.0,106.495469,108.794479,106.409968,108.290977,55.633496,1.328858,-0.004264,0.000000,0.016059,0.017677
-3,27598920.0,108.100976,109.031980,108.015476,108.214976,55.382828,-0.181550,0.015076,0.015088,0.002183,-0.000702
-2,29881477.0,107.958475,108.224476,106.504969,106.894471,51.076636,0.082705,-0.001318,-0.013984,-0.007406,-0.012203
-1,41403351.0,107.179472,107.473973,104.699961,104.861462,45.244023,0.385586,-0.007216,-0.016948,-0.006935,-0.019019
0,53204626.0,105.820966,105.868466,101.982949,103.863957,42.669377,0.285032,-0.012675,-0.025950,-0.014939,-0.009513


In [286]:
# normalize columns from dollars to "1" - turns the columns into ratios compared to day N (index 0)

cols_to_normalize_to_1_for_day_0 = ['AdjVolume', 'AdjOpen', 'AdjLow', 'AdjHigh', 'AdjClose']

df_extract.loc[:, cols_to_normalize_to_1_for_day_0] = df_extract.loc[:, cols_to_normalize_to_1_for_day_0] / df_extract.loc[0, cols_to_normalize_to_1_for_day_0]

df_extract

Attributes,AdjVolume,AdjOpen,AdjHigh,AdjLow,AdjClose,AdjCloseRSI,AdjVolume_chg,AdjOpen_chg,AdjLow_chg,AdjHigh_chg,AdjClose_chg
row_index,,,,,,,,,,,
-4,0.633797,1.006374,1.027638,1.043409,1.042623,55.633496,1.328858,-0.004264,0.000000,0.016059,0.017677
-3,0.518732,1.021546,1.029882,1.059152,1.041892,55.382828,-0.181550,0.015076,0.015088,0.002183,-0.000702
-2,0.561633,1.020199,1.022254,1.044341,1.029178,51.076636,0.082705,-0.001318,-0.013984,-0.007406,-0.012203
-1,0.778191,1.012838,1.015165,1.026642,1.009604,45.244023,0.385586,-0.007216,-0.016948,-0.006935,-0.019019
0,1.000000,1.000000,1.000000,1.000000,1.000000,42.669377,0.285032,-0.012675,-0.025950,-0.014939,-0.009513


In [190]:
# unstack and make it tall (ie unpivot)
df_extract = df_extract.unstack().reset_index()
df_extract

,Attributes,row_index,0
0,AdjCloseRSI,-4,55.633496
1,AdjCloseRSI,-3,55.382828
2,AdjCloseRSI,-2,51.076636
3,AdjCloseRSI,-1,45.244023
4,AdjCloseRSI,0,42.669377
5,AdjVolume_chg,-4,1.328858
6,AdjVolume_chg,-3,-0.181550
7,AdjVolume_chg,-2,0.082705
8,AdjVolume_chg,-1,0.385586
9,AdjVolume_chg,0,0.285032


In [191]:
# create new column with combined field names of attribute and index
# eg: AdjClose-1 for the adjusted close of day N-1 or AdjHigh-4 for the adjusted High of day N-4
df_extract['Attribute-index'] = df_extract['Attributes'] + df_extract['row_index'].apply(str)
# then drop Attributes and row_index columns since they are not needed anymore
df_extract.drop(columns=['Attributes', 'row_index'], inplace=True)
df_extract

,0,Attribute-index
0,55.633496,AdjCloseRSI-4
1,55.382828,AdjCloseRSI-3
2,51.076636,AdjCloseRSI-2
3,45.244023,AdjCloseRSI-1
4,42.669377,AdjCloseRSI0
5,1.328858,AdjVolume_chg-4
6,-0.181550,AdjVolume_chg-3
7,0.082705,AdjVolume_chg-2
8,0.385586,AdjVolume_chg-1
9,0.285032,AdjVolume_chg0


In [192]:
# set index one and transpose
target_row = df_extract.set_index('Attribute-index').T
# we now have one row of data that represents the prior n_feature_days worth of data:
target_row

Attribute-index,AdjCloseRSI-4,AdjCloseRSI-3,AdjCloseRSI-2,AdjCloseRSI-1,AdjCloseRSI0,AdjVolume_chg-4,AdjVolume_chg-3,AdjVolume_chg-2,AdjVolume_chg-1,AdjVolume_chg0,...,AdjHigh_chg-4,AdjHigh_chg-3,AdjHigh_chg-2,AdjHigh_chg-1,AdjHigh_chg0,AdjClose_chg-4,AdjClose_chg-3,AdjClose_chg-2,AdjClose_chg-1,AdjClose_chg0
0,55.633496,55.382828,51.076636,45.244023,42.669377,1.328858,-0.18155,0.082705,0.385586,0.285032,...,0.016059,0.002183,-0.007406,-0.006935,-0.014939,0.017677,-0.000702,-0.012203,-0.019019,-0.009513


In [193]:
df_X_base_data.index.to_list()[504]

Timestamp('2015-01-02 00:00:00')

In [194]:
target_row['Index'] = df_X_base_data.index.to_list()[504]
target_row = target_row.set_index('Index')
target_row

Attribute-index,AdjCloseRSI-4,AdjCloseRSI-3,AdjCloseRSI-2,AdjCloseRSI-1,AdjCloseRSI0,AdjVolume_chg-4,AdjVolume_chg-3,AdjVolume_chg-2,AdjVolume_chg-1,AdjVolume_chg0,...,AdjHigh_chg-4,AdjHigh_chg-3,AdjHigh_chg-2,AdjHigh_chg-1,AdjHigh_chg0,AdjClose_chg-4,AdjClose_chg-3,AdjClose_chg-2,AdjClose_chg-1,AdjClose_chg0
Index,,,,,,,,,,,,,,,,,,,,,
2015-01-02,55.633496,55.382828,51.076636,45.244023,42.669377,1.328858,-0.18155,0.082705,0.385586,0.285032,...,0.016059,0.002183,-0.007406,-0.006935,-0.014939,0.017677,-0.000702,-0.012203,-0.019019,-0.009513


#### --- done building the pieces, now implement in one loop to cycle through all rows of df ---

In [288]:
def create_feature_cols_df(df_X_base_data,
                           n_days_features=n_days_features,
                           cols_to_normalize_to_1_for_day_0 = ['AdjVolume', 'AdjOpen', 'AdjLow', 'AdjHigh', 'AdjClose']):
    '''
    Take dataframe with date index (sorted increasing time) with multiple columns and return a new wider dataframe
    where the rows for the last n_days_features have been pivoted into additional columns
    Input:
        df_X_base_data - dataframe with date index
        n_days_features - number of prior days that are pivoted into the rows
        cols_to_normalize_to_1_for_day_0 - columns that will be normalized for day N. eg prices, volumes
    Output:
        df_X - datafram that has length of df_X_base_data.shape[0] - n_days_features and more columns than df_X_base_data
    '''
    
    df_X = pd.DataFrame()
    
    # total lenght of df
    n_data_points = df_X_base_data.shape[0]
    
    # cycle through each row of df, start at n_days_features-1 because we wouldn't have enough history for first rows
    for i in tqdm(range(n_days_features, n_data_points+1), desc='reshaping data into feature rows'):
        # i contains the rows number of df

        df_extract = df_X_base_data.iloc[i-n_days_features:i, :]

        # pull out n_days_features of rows from current position
        df_extract = df_X_base_data.iloc[i-n_days_features:i, :].copy()

        # change the index to be "days into the past" - eg current day is 0, prior day is -1, ...
        df_extract.loc[:, 'row_index'] = range(-n_days_features+1, 1)

        # make this the new index
        df_extract.set_index('row_index', inplace=True)
        
        # normalize columns from dollars to "1" - turns the columns into ratios compared to day N (index 0)
        df_extract.loc[:, cols_to_normalize_to_1_for_day_0] = df_extract.loc[:, cols_to_normalize_to_1_for_day_0] / df_extract.loc[0, cols_to_normalize_to_1_for_day_0]

        # unstack and make it tall (ie unpivot)
        df_extract = df_extract.unstack().reset_index()

        # create new column with combined field names of attribute and index
        # eg: AdjClose-1 for the adjusted close of day N-1 or AdjHigh-4 for the adjusted High of day N-4
        df_extract['Attribute-index'] = df_extract['Attributes'] + df_extract['row_index'].apply(str)
        # then drop Attributes and row_index columns since they are not needed anymore
        df_extract.drop(columns=['Attributes', 'row_index'], inplace=True)

        # set index one and transpose
        target_row = df_extract.set_index('Attribute-index').T
        # we now have one row of data that represents the prior n_feature_days worth of data

        # fill in the target_row index with the date from the index of the source dataframe df_X_base_data (ie, day N)
        target_row['Index'] = df_X_base_data.index.to_list()[i-1] # zero-indexed so need minus 1
        target_row = target_row.set_index('Index')

        df_X = df_X.append(target_row)
        
    return df_X

# split df into features and target (column setup_for_profitable_trade)
df_X_base_data = df.drop(columns=['setup_for_profitable_trade'])
df_y = df['setup_for_profitable_trade']

# feature engineering: instead of dollars/absolute values, calculate change from one day to next
cols_set_vals_to_change = ['AdjVolume', 'AdjOpen', 'AdjLow', 'AdjHigh', 'AdjClose']
df_X_base_data = feat_eng_changes_values_to_change(df_X_base_data, cols_set_vals_to_change, delete_original_cols=False)

# create wide features matrix that includes prior days' data as columns
df_X = create_feature_cols_df(df_X_base_data,
                              cols_to_normalize_to_1_for_day_0 = ['AdjVolume', 'AdjOpen', 'AdjLow', 'AdjHigh', 'AdjClose'])
    
df_X.tail()

reshaping data into feature rows: 100%|████| 1310/1310 [00:34<00:00, 39.49it/s]


Attribute-index,AdjVolume-4,AdjVolume-3,AdjVolume-2,AdjVolume-1,AdjVolume0,AdjOpen-4,AdjOpen-3,AdjOpen-2,AdjOpen-1,AdjOpen0,...,AdjHigh_chg-4,AdjHigh_chg-3,AdjHigh_chg-2,AdjHigh_chg-1,AdjHigh_chg0,AdjClose_chg-4,AdjClose_chg-3,AdjClose_chg-2,AdjClose_chg-1,AdjClose_chg0
Index,,,,,,,,,,,,,,,,,,,,,
2018-03-19,0.950120,0.886320,0.688455,1.122902,1.0,1.029720,1.016919,1.006655,1.007501,1.0,...,0.006086,-0.016240,-0.001551,-0.006214,-0.009212,-0.009630,-0.008501,0.001177,-0.003526,-0.015279
2018-03-20,1.505406,1.169334,1.907237,1.698490,1.0,1.028989,1.018603,1.019459,1.011869,1.0,...,-0.016240,-0.001551,-0.006214,-0.009212,-0.003775,-0.008501,0.001177,-0.003526,-0.015279,-0.000342
2018-03-21,0.640745,1.045084,0.930699,0.547957,1.0,1.019767,1.020624,1.013026,1.001143,1.0,...,-0.001551,-0.006214,-0.009212,-0.003775,-0.009672,0.001177,-0.003526,-0.015279,-0.000342,-0.022655
2018-03-22,0.897332,0.799119,0.470488,0.858622,1.0,1.050882,1.043059,1.030824,1.029647,1.0,...,-0.006214,-0.009212,-0.003775,-0.009672,-0.013764,-0.003526,-0.015279,-0.000342,-0.022655,-0.014159
2018-03-23,0.815045,0.479864,0.875734,1.019929,1.0,1.053032,1.040679,1.039492,1.009561,1.0,...,-0.009212,-0.003775,-0.009672,-0.013764,-0.015983,-0.015279,-0.000342,-0.022655,-0.014159,-0.023128


### Feature Engineering - Date information

In [291]:
def feat_eng_append_date_index_content(df):
    '''
    Assumes that the df index is date-time. Bolts on additional columns about the date
    '''

    df['year'] = df.index.year.values
    df['month'] = df.index.month.values
    df['week'] = df.index.week.values
    df['weekday'] = df.index.weekday.values
    df['day'] = df.index.day.values
    df['year'] = df.index.year.values
    df['year'] = df.index.year.values
    
    return df
    
df_X = feat_eng_append_date_index_content(df_X)
df_X.tail()

Attribute-index,AdjVolume-4,AdjVolume-3,AdjVolume-2,AdjVolume-1,AdjVolume0,AdjOpen-4,AdjOpen-3,AdjOpen-2,AdjOpen-1,AdjOpen0,AdjHigh-4,AdjHigh-3,AdjHigh-2,AdjHigh-1,AdjHigh0,AdjLow-4,AdjLow-3,AdjLow-2,AdjLow-1,AdjLow0,AdjClose-4,AdjClose-3,AdjClose-2,AdjClose-1,AdjClose0,AdjVolume_chg-4,AdjVolume_chg-3,AdjVolume_chg-2,AdjVolume_chg-1,AdjVolume_chg0,AdjOpen_chg-4,AdjOpen_chg-3,AdjOpen_chg-2,AdjOpen_chg-1,AdjOpen_chg0,AdjLow_chg-4,AdjLow_chg-3,AdjLow_chg-2,AdjLow_chg-1,AdjLow_chg0,AdjHigh_chg-4,AdjHigh_chg-3,AdjHigh_chg-2,AdjHigh_chg-1,AdjHigh_chg0,AdjClose_chg-4,AdjClose_chg-3,AdjClose_chg-2,AdjClose_chg-1,AdjClose_chg0,year,month,week,weekday,day
Index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2018-03-19,0.950120,0.886320,0.688455,1.122902,1.0,1.029720,1.016919,1.006655,1.007501,1.0,1.033978,1.017186,1.015608,1.009297,1.0,1.032132,1.023897,1.025395,1.022803,1.0,1.026640,1.017912,1.019110,1.015516,1.0,-0.027671,-0.067149,-0.223243,0.631046,-0.109450,0.012757,-0.012432,-0.010093,0.000840,-0.007445,-0.005383,-0.007978,0.001463,-0.002528,-0.022295,0.006086,-0.016240,-0.001551,-0.006214,-0.009212,-0.009630,-0.008501,0.001177,-0.003526,-0.015279,2018,3,12,0,19
2018-03-20,1.505406,1.169334,1.907237,1.698490,1.0,1.028989,1.018603,1.019459,1.011869,1.0,1.021041,1.019457,1.013122,1.003790,1.0,1.016406,1.017892,1.015320,0.992683,1.0,1.018261,1.019459,1.015864,1.000342,1.0,-0.067149,-0.223243,0.631046,-0.109450,-0.411242,-0.012432,-0.010093,0.000840,-0.007445,-0.011730,-0.007978,0.001463,-0.002528,-0.022295,0.007371,-0.016240,-0.001551,-0.006214,-0.009212,-0.003775,-0.008501,0.001177,-0.003526,-0.015279,-0.000342,2018,3,12,1,20
2018-03-21,0.640745,1.045084,0.930699,0.547957,1.0,1.019767,1.020624,1.013026,1.001143,1.0,1.029413,1.023017,1.013593,1.009766,1.0,1.039765,1.037137,1.014014,1.021488,1.0,1.043090,1.039411,1.023530,1.023180,1.0,-0.223243,0.631046,-0.109450,-0.411242,0.824960,-0.010093,0.000840,-0.007445,-0.011730,-0.001141,0.001463,-0.002528,-0.022295,0.007371,-0.021036,-0.001551,-0.006214,-0.009212,-0.003775,-0.009672,0.001177,-0.003526,-0.015279,-0.000342,-0.022655,2018,3,12,2,21
2018-03-22,0.897332,0.799119,0.470488,0.858622,1.0,1.050882,1.043059,1.030824,1.029647,1.0,1.037294,1.027739,1.023859,1.013956,1.0,1.053499,1.030012,1.037604,1.015777,1.0,1.054340,1.038230,1.037875,1.014362,1.0,0.631046,-0.109450,-0.411242,0.824960,0.164657,0.000840,-0.007445,-0.011730,-0.001141,-0.028793,-0.002528,-0.022295,0.007371,-0.021036,-0.015532,-0.006214,-0.009212,-0.003775,-0.009672,-0.013764,-0.003526,-0.015279,-0.000342,-0.022655,-0.014159,2018,3,12,3,22
2018-03-23,0.815045,0.479864,0.875734,1.019929,1.0,1.053032,1.040679,1.039492,1.009561,1.0,1.044433,1.040490,1.030426,1.016243,1.0,1.052868,1.060628,1.038317,1.022190,1.0,1.062811,1.062447,1.038378,1.023675,1.0,-0.109450,-0.411242,0.824960,0.164657,-0.019540,-0.007445,-0.011730,-0.001141,-0.028793,-0.009471,-0.022295,0.007371,-0.021036,-0.015532,-0.021708,-0.009212,-0.003775,-0.009672,-0.013764,-0.015983,-0.015279,-0.000342,-0.022655,-0.014159,-0.023128,2018,3,12,4,23


##### just testing aroung: can also use only one or a few columns and then join the dataframes back together

In [ ]:
df_XO = create_feature_cols_df(df_X_base_data[['AdjOpen']])
df_XO.tail()

In [ ]:
df_X.join(df_XO).tail()

## Machine Learning

### Build Model

In [197]:
# need to remove first n_days_features rows - they were eliminated during build of df_X and sizes need to match
df_y = df_y.iloc[n_days_features-1:]

In [198]:
# remove all rows that have any NaNs in them - they come from technical indicators or the reshaping and we just don't have 
# any good strategy for imputation other than starting with more time series data
df_X = df_X.loc[df_X.notnull().all(axis=1), :]
df_X.shape

(1309, 30)

In [199]:
# now we need to keep the same rows that we kept in df_X in df_y. We'll use the index to filter
df_y = df_y.loc[df_X.index]
df_y.shape

(1309,)

In [233]:
X_train, X_test, y_train, y_test = train_test_split(df_X, df_y, test_size=0.2, shuffle=False)

In [234]:
pipeline = Pipeline([
    ('randomForest', RandomForestClassifier())
])
pipeline.get_params()

{'memory': None,
 'steps': [('randomForest',
   RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
               oob_score=False, random_state=None, verbose=0,
               warm_start=False))],
 'randomForest': RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
             max_depth=None, max_features='auto', max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
             oob_score=False, random_state=None, verbose=0,
             warm_start=False),
 'randomForest__bootstrap': True,
 'randomForest__c

In [235]:
pipeline.fit(X_train, y_train)

C:\Users\Lindstaedt\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Pipeline(memory=None,
     steps=[('randomForest', RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))])

In [236]:
y_pred = pipeline.predict(X_test)

In [237]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

       False       0.66      0.77      0.71       175
        True       0.31      0.21      0.25        87

   micro avg       0.58      0.58      0.58       262
   macro avg       0.49      0.49      0.48       262
weighted avg       0.55      0.58      0.56       262



In [238]:
accuracy_score(y_test, y_pred)

0.583969465648855

In [239]:
y_pred.shape

(262,)

In [240]:
y_pred

array([False, False, False,  True, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False,  True,  True, False,  True, False, False, False, False,
       False, False, False, False,  True, False, False, False,  True,
       False, False, False, False, False, False, False, False,  True,
        True,  True,  True, False, False, False, False,  True, False,
        True, False,  True, False, False, False, False,  True, False,
       False, False, False, False, False, False,  True, False, False,
       False, False,  True, False, False, False, False, False,  True,
       False, False, False, False, False,  True, False, False, False,
       False, False, False, False, False, False, False, False,  True,
       False,  True, False,  True, False, False, False, False, False,
       False, False, False, False,  True,  True,  True, False, False,
       False,  True, False,  True,  True, False, False, False, False,
       False, False,

### Improve RandomForest model

In [241]:
parameters = {
    'randomForest__min_samples_leaf': [1, 2, 5, 10]
}

cv = GridSearchCV(pipeline, param_grid=parameters, cv=5, scoring='precision', n_jobs=-1, verbose=6)
cv.fit(X_train, y_train)

Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:    0.7s finished


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('randomForest', RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))]),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'randomForest__min_samples_leaf': [1, 2, 5, 10]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='precision', verbose=6)

In [242]:
cv.best_params_

{'randomForest__min_samples_leaf': 2}

In [243]:
y_pred = cv.predict(X_test)

In [244]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

       False       0.68      0.82      0.74       175
        True       0.38      0.23      0.29        87

   micro avg       0.62      0.62      0.62       262
   macro avg       0.53      0.52      0.52       262
weighted avg       0.58      0.62      0.59       262



In [245]:
accuracy_score(y_test, y_pred)

0.6221374045801527

In [246]:
confusion_matrix(y_test, y_pred, labels=[False, True])

array([[143,  32],
       [ 67,  20]], dtype=int64)

In [247]:
y_pred

array([False, False, False, False, False,  True, False, False, False,
        True, False, False,  True, False, False, False, False, False,
       False, False,  True, False, False, False, False, False, False,
       False, False,  True, False, False, False, False, False, False,
       False, False, False, False,  True, False, False, False, False,
        True,  True, False, False, False, False, False, False, False,
       False, False, False, False,  True, False, False, False, False,
       False, False, False,  True, False, False,  True, False, False,
        True,  True, False,  True, False, False, False, False,  True,
       False, False, False, False,  True, False, False, False, False,
        True, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False,  True, False, False,
       False, False, False, False,  True, False, False, False,  True,
       False, False, False, False, False, False,  True, False, False,
       False, False,

In [248]:
X_test.join(y_test)

,AdjVolume_chg-4,AdjVolume_chg-3,AdjVolume_chg-2,AdjVolume_chg-1,AdjVolume_chg0,AdjOpen_chg-4,AdjOpen_chg-3,AdjOpen_chg-2,AdjOpen_chg-1,AdjOpen_chg0,...,AdjClose_chg-3,AdjClose_chg-2,AdjClose_chg-1,AdjClose_chg0,year,month,week,weekday,day,setup_for_profitable_trade
Index,,,,,,,,,,,,,,,,,,,,,
2017-03-08,-0.280206,-0.177020,0.008295,-0.197873,0.072275,0.015302,-0.008714,0.004215,-0.002188,-0.000791,...,0.005901,-0.003148,0.001292,-0.003727,2017,3,10,2,8,False
2017-03-09,-0.177020,0.008295,-0.197873,0.072275,0.184349,-0.008714,0.004215,-0.002188,-0.000791,-0.001511,...,-0.003148,0.001292,-0.003727,-0.002302,2017,3,10,3,9,False
2017-03-10,0.008295,-0.197873,0.072275,0.184349,-0.114782,0.004215,-0.002188,-0.000791,-0.001511,0.003676,...,0.001292,-0.003727,-0.002302,0.003317,2017,3,10,4,10,False
2017-03-13,-0.197873,0.072275,0.184349,-0.114782,-0.111717,-0.002188,-0.000791,-0.001511,0.003676,-0.002873,...,-0.003727,-0.002302,0.003317,0.000431,2017,3,11,0,13,False
2017-03-14,0.072275,0.184349,-0.114782,-0.111717,-0.121265,-0.000791,-0.001511,0.003676,-0.002873,0.003241,...,-0.002302,0.003317,0.000431,-0.001509,2017,3,11,1,14,True
2017-03-15,0.184349,-0.114782,-0.111717,-0.121265,0.678207,-0.001511,0.003676,-0.002873,0.003241,0.000790,...,0.003317,0.000431,-0.001509,0.010576,2017,3,11,2,15,False
2017-03-16,-0.114782,-0.111717,-0.121265,0.678207,-0.251434,0.003676,-0.002873,0.003241,0.000790,0.009397,...,0.000431,-0.001509,0.010576,0.001637,2017,3,11,3,16,False
2017-03-17,-0.111717,-0.121265,0.678207,-0.251434,1.281872,-0.002873,0.003241,0.000790,0.009397,0.001990,...,-0.001509,0.010576,0.001637,-0.004975,2017,3,11,4,17,True
2017-03-20,-0.121265,0.678207,-0.251434,1.281872,-0.509125,0.003241,0.000790,0.009397,0.001990,-0.004255,...,0.010576,0.001637,-0.004975,0.010501,2017,3,12,0,20,False


## next: add backtesting

## use TPOT
## add column for stock
## add additional technical indicators
## add crossover metrics